# accessing turbulence isotropic cube files on filedb cluster from sciserver
* created volume container (turb) showing all /turb folders on filedb system
* access to turbinfo for metadata about these files
 * requires copying DataPath table
* morton curve code pip installed in sciserver container
* some special code to take into account 8x8x8 blobs with z-y-x ordering

In [ ]:
%pip install morton-py

In [18]:
import os
import sys
import math
import time
import morton
import struct
import numpy as np
import SciServer.CasJobs as cj

class IsoCube:
    def __init__(self, cube_num, cube_dimensions = 3, cube_subtitle = ''):
        # setting up Morton curve
        # cube size
        # the assumed name of the dataset names are "isotropic{N}" (e.g. "isotropic8192")
        self.N = cube_num
        # if the turbulence database has a sub title (e.g. "isotropic1024fine" or "isotropic1024coarse"), specify
        # the subtitle here.  otherwise leave this as an empty string.
        self.N_subtitle = cube_subtitle
        bits = int(math.log(self.N,2))
        self.mortoncurve = morton.Morton(dimensions = cube_dimensions, bits = bits)
        self.initcache()
        
    def initcache(self):
        # read SQL metadata for all of the turbulence data files into the cache
        sql = f"""
        select dbm.ProductionMachineName
        , dbm.ProductionDatabaseName
        , dbm.minLim, dbm.maxLim
        , dbm.minTime, dbm.maxTime
        , dp.path
        from databasemap dbm
           join datapath{str(self.N) + self.N_subtitle} dp
             on dp.datasetid=dbm.datasetid
           and dp.productionmachinename=dbm.productionmachinename
           and dp.ProductionDatabaseName=dbm.ProductionDatabaseName
        where dbm.datasetname = 'isotropic{str(self.N) + self.N_subtitle}'
        order by minlim
        """
        df = cj.executeQuery(sql, "turbinfo")
        
        x, y, z = self.mortoncurve.unpack(df['minLim'].values)
        df['x_min'] = x
        df['y_min'] = y
        df['z_min'] = z
        
        x, y, z = self.mortoncurve.unpack(df['maxLim'].values)
        df['x_max'] = x
        df['y_max'] = y 
        df['z_max'] = z
        
        self.cache = df
    
    # defines some helper functions, all hardcoded (double-check this when other datasets are available)
    def parseCornerPoints(self, x_min, x_max, y_min, y_max, z_min, z_max):
        # only points 1, 2, 4, and 5 are required for finding the correct sub-boxes.
        # corner 1 is the bottom left back side origin point.
        # corner 2 is the bottom right back side corner point (same as corner 1 except at the maximum x-position).
        # corner 4 is the bottom left front side corner point (same as corner 1 except at the maximum y-positon).
        # corner 5 is the top left back corner point (same as corner 1 except at the maximum z-positon).
        # corners 2, 3, and 4 travel around the bottom plane of the box clockwise from corner 1.
        # corners 6, 7, and 8 travel around the top plane of the box clockwise from corner 5.
        c1 = (x_min, y_min, z_min)
        c2 = (x_max, y_min, z_min)
        #c3 = (x_max, y_max, z_min)
        c4 = (x_min, y_max, z_min)
        c5 = (x_min, y_min, z_max)
        #c6 = (x_max, y_min, z_max)
        #c7 = (x_max, y_max, z_max)
        #c8 = (x_min, y_max, z_max)
        
        corner_points = (c1, c2, c4, c5)
        
        return corner_points
        
    def getFilesForCornerPoints(self, x_range, y_range, z_range, var, timepoint):
        # define the corner points.
        x_min = x_range[0]; x_max = x_range[1];
        y_min = y_range[0]; y_max = y_range[1];
        z_min = z_range[0]; z_max = z_range[1];
        
        # retrieve the corner points.
        c_points = self.parseCornerPoints(x_min, x_max, y_min, y_max, z_min, z_max)
        
        database_files = []
        
        # only points 1, 2, 4, and 5 are required for finding the correct sub-boxes.
        c1_info = self.getFileForPoint(c_points[0][0], c_points[0][1], c_points[0][2], var, timepoint)
        c1_file = c1_info[0]
        database_files.append(c1_file)
        
        c2_info = self.getFileForPoint(c_points[1][0], c_points[1][1], c_points[1][2], var, timepoint)
        c2_file = c2_info[0]
        database_files.append(c2_file)
        
        c4_info = self.getFileForPoint(c_points[2][0], c_points[2][1], c_points[2][2], var, timepoint)
        c4_file = c4_info[0]
        database_files.append(c4_file)
        
        c5_info = self.getFileForPoint(c_points[3][0], c_points[3][1], c_points[3][2], var, timepoint)
        c5_file = c5_info[0]
        database_files.append(c5_file)
        
        return database_files
    
    def findSubBoxEndPoint(self, axis_range, datapoint, axis_position, db_file_comparison, var, timepoint):
        # placeholder end point value. 
        end_point = -1
        # if the difference between the axis range end points is <= to this value, then the end_point
        # has been found.
        axis_range_difference = 2
        
        end_point_found = False
        while not end_point_found:
            mid_point = math.floor((axis_range[0] + axis_range[1]) / 2)
            
            # stops recursively shrinking the box once the difference between the two end points is <= axis_range_difference.
            if (axis_range[1] - axis_range[0]) <= axis_range_difference:
                end_point_found = True
            
            # updates the datapoint to the new mid point.
            datapoint[axis_position] = mid_point
            
            # gets the db file for the new datapoint.
            datapoint_info = self.getFileForPoint(datapoint[0], datapoint[1], datapoint[2], var, timepoint)
            datapoint_file = datapoint_info[0]
            
            # compares the db file for datapoint to the origin point.
            if datapoint_file == db_file_comparison:
                end_point = mid_point
                axis_range[0] = mid_point
            else:
                end_point = mid_point - 1
                axis_range[1] = mid_point
            
            # used for checking that there were no redundant calculations
            #print(f'midpoint = {mid_point}')
            #print(f'endpoint = {end_point}')
            #print('-')
                
        return end_point
    
    def identifySingleDatabaseFileSubBoxes(self, x_range, y_range, z_range, var, timepoint):
        # initially assumes the user specified box contains points in different files. the boxes will be split up until all the points
        # in each box are from a single database file.
        boxes_to_check = [(x_range, y_range, z_range)]
        single_file_boxes = {}
        
        while len(boxes_to_check) != 0:
            for box in reversed(boxes_to_check):
                db_files = self.getFilesForCornerPoints(box[0], box[1], box[2], var, timepoint)
                num_db_files = len(set(db_files))
                
                if num_db_files == 1:
                    single_file_boxes[list(set(db_files))[0]] = box 
                elif db_files[0] != db_files[1]:
                    # this means that the x_range was sufficiently large such that all of the points were
                    # not contained in a singular database file.  i.e. the database files were different for
                    # corners 1 and 2.  the data x_range will now be split in half to create 2 sub-boxes for checking.
                    
                    # this value is specified as 0 because the x-axis index is 0.  this is used for determing which 
                    # point (X, Y, or Z) the midpoint is going to be tested for.  in this case, this section of code
                    # is adjusting only the x-axis.
                    axis_position = 0
                    # stores the c1 corner point (X, Y, Z) of the box to be used for finding the first box end point
                    # when shrinking the x-axis into sub-boxes.
                    datapoint = [box[0][0], box[1][0], box[2][0]]
                    # which axis is sub-divided, in this case it is the x-axis.
                    axis_range = list(box[0])
                    # determine where the end x-axis point is for the first sub-box.
                    first_box_end_point = self.findSubBoxEndPoint(axis_range, datapoint, axis_position, db_files[0], \
                                                                  var, timepoint)
                    
                    first_sub_box = [[box[0][0], first_box_end_point], box[1], box[2]]
                    second_sub_box = [[first_box_end_point + 1, box[0][1]], box[1], box[2]]
                    
                    boxes_to_check.append(second_sub_box)
                    boxes_to_check.append(first_sub_box)
                elif db_files[0] != db_files[2]:
                    # this means that the y_range was sufficiently large such that all of the points were
                    # not contained in a singular database file.  i.e. the database files were different for
                    # corners 1 and 4.  the data y_range will now be split in half to create 2 sub-boxes for checking.
                    
                    # this value is specified as 1 because the y-axis index is 1.  this is used for determing which 
                    # point (X, Y, or Z) the midpoint is going to be tested for.  in this case, this section of code
                    # is adjusting only the y-axis.
                    axis_position = 1
                    # stores the c1 corner point (X, Y, Z) of the box to be used for finding the first box end point 
                    # when shrinking the y-axis into sub-boxes.
                    datapoint = [box[0][0], box[1][0], box[2][0]]
                    # which axis is sub-divided, in this case it is the y-axis.
                    axis_range = list(box[1])
                    # determine where the end y-axis point is for the first sub-box.
                    first_box_end_point = self.findSubBoxEndPoint(axis_range, datapoint, axis_position, db_files[0], \
                                                                  var, timepoint)
                    
                    first_sub_box = [box[0], [box[1][0], first_box_end_point], box[2]]
                    second_sub_box = [box[0], [first_box_end_point + 1, box[1][1]], box[2]]
                    
                    boxes_to_check.append(second_sub_box)
                    boxes_to_check.append(first_sub_box)
                elif db_files[0] != db_files[3]:
                    # this means that the z_range was sufficiently large such that all of the points were
                    # not contained in a singular database file.  i.e. the database files were different for
                    # corners 1 and 5.  the data z_range will now be split in half to create 2 sub-boxes for checking.
                    
                    # this value is specified as 2 because the z-axis index is 2.  this is used for determing which 
                    # point (X, Y, or Z) the midpoint is going to be tested for.  in this case, this section of code
                    # is adjusting only the z-axis.
                    axis_position = 2
                    # stores the c1 corner point (X, Y, Z) of the box to be used for finding the first box end point 
                    # when shrinking the z-axis into sub-boxes.
                    datapoint = [box[0][0], box[1][0], box[2][0]]
                    # which axis is sub-divided, in this case it is the z-axis.
                    axis_range = list(box[2])
                    # determine where the end z-axis point is for the first sub-box.
                    first_box_end_point = self.findSubBoxEndPoint(axis_range, datapoint, axis_position, db_files[0], \
                                                                  var, timepoint)
                    
                    first_sub_box = [box[0], box[1], [box[2][0], first_box_end_point]]
                    second_sub_box = [box[0], box[1], [first_box_end_point + 1, box[2][1]]]
                    
                    boxes_to_check.append(second_sub_box)
                    boxes_to_check.append(first_sub_box)
                    
                # either the original box is fully contained in a single database file, or it is in more than one.
                # in either case, the box should be removed from further checking.  the sub-boxes will be checked
                # if necessary.
                boxes_to_check.remove(box)
            
        return single_file_boxes
    
    def boxesContained(self, sub_box, user_box):
        contained = False
        # checks if the sub-divided box is fully contained within the user-specified box.
        if (sub_box[0][0] >= user_box[0][0] and sub_box[0][1] <= user_box[0][1]) and \
            (sub_box[1][0] >= user_box[1][0] and sub_box[1][1] <= user_box[1][1]) and \
            (sub_box[2][0] >= user_box[2][0] and sub_box[2][1] <= user_box[2][1]):
            contained = True
        
        return contained
    
    def boxesOverlap(self, sub_box, user_box):
        overlap = False
        # checks if the sub-divided box and the user-specified box overlap on all 3 axes
        if (sub_box[0][0] <= user_box[0][1] and user_box[0][0] <= sub_box[0][1]) and \
            (sub_box[1][0] <= user_box[1][1] and user_box[1][0] <= sub_box[1][1]) and \
            (sub_box[2][0] <= user_box[2][1] and user_box[2][0] <= sub_box[2][1]):
            overlap = True
            
        return overlap
    
    def recursiveSubBoxesInFile(self, box, user_db_box, sub_boxes):
        # recursively sub-divides the database file cube until the entire user-specified box is mapped by morton cubes
        box_x_range = box[0]
        box_y_range = box[1]
        box_z_range = box[2]
        
        # checks if the sub-box corner points are all inside the portion of the user-specified box in the database file.
        box_fully_contained = self.boxesContained(box, user_db_box)
        if box_fully_contained:
            # converts the box (X, Y, Z) minimum and maximum points to morton indices
            morton_index_min = self.mortoncurve.pack(box[0][0], box[1][0], box[2][0])
            morton_index_max = self.mortoncurve.pack(box[0][1], box[1][1], box[2][1])
            
            if sub_boxes == list():
                sub_boxes.append([morton_index_min, morton_index_max])
            else:
                # check if the most recent sub-box maximum is 1 index less than the new sub-box minimum.  if so, then extend the range of the previous sub-box
                # morton maximum to stitch these two boxes together.
                if sub_boxes[-1][1] == (morton_index_min - 1):
                    sub_boxes[-1][1] = morton_index_max
                else:
                    sub_boxes.append([morton_index_min, morton_index_max])
            
            return
        else:
            box_partially_contained = self.boxesOverlap(box, user_db_box)
            if box_partially_contained:
                # sub-divide the box into 8 sub-cubes (divide the x-, y-, and z- axes in half) and recursively check each box if it is inside the user-specified
                # box, if necessary.
                box_x_range_midpoint = math.floor((box_x_range[0] + box_x_range[1]) / 2)
                box_y_range_midpoint = math.floor((box_y_range[0] + box_y_range[1]) / 2)
                box_z_range_midpoint = math.floor((box_z_range[0] + box_z_range[1]) / 2)
                
                # ordering sub-boxes 1-8 below in this order maintains the morton-curve index structure, such that the minimum (X, Y, Z) morton index 
                # for a new box only needs to be compared to the last sub-boxes' maximum (X, Y, Z) morton index to see if they can be stitched together.
                
                # new_sub_box_1 is the sub-box bounded by [x_min, x_midpoint], [y_min, y_midpoint], and [z_min, z_midpoint]
                # new_sub_box_2 is the sub-box bounded by [x_midpoint + 1, x_max], [y_min, y_midpoint], and [z_min, z_midpoint]
                # new_sub_box_3 is the sub-box bounded by [x_min, x_midpoint], [y_midpoint + 1, y_max], and [z_min, z_midpoint]
                # new_sub_box_4 is the sub-box bounded by [x_midpoint + 1, x_max], [y_midpoint + 1, y_max], and [z_min, z_midpoint]
                new_sub_box_1 = [[box_x_range[0], box_x_range_midpoint], [box_y_range[0], box_y_range_midpoint], [box_z_range[0], box_z_range_midpoint]]
                new_sub_box_2 = [[box_x_range_midpoint + 1, box_x_range[1]], [box_y_range[0], box_y_range_midpoint], [box_z_range[0], box_z_range_midpoint]]
                new_sub_box_3 = [[box_x_range[0], box_x_range_midpoint], [box_y_range_midpoint + 1, box_y_range[1]], [box_z_range[0], box_z_range_midpoint]]
                new_sub_box_4 = [[box_x_range_midpoint + 1, box_x_range[1]], [box_y_range_midpoint + 1, box_y_range[1]], [box_z_range[0], box_z_range_midpoint]]
                
                # new_sub_box_5 is the sub-box bounded by [x_min, x_midpoint], [y_min, y_midpoint], and [z_midpoint + 1, z_max]
                # new_sub_box_6 is the sub-box bounded by [x_midpoint + 1, x_max], [y_min, y_midpoint], and [z_midpoint + 1, z_max]
                # new_sub_box_7 is the sub-box bounded by [x_min, x_midpoint], [y_midpoint + 1, y_max], and [z_midpoint + 1, z_max]
                # new_sub_box_8 is the sub-box bounded by [x_midpoint + 1, x_max], [y_midpoint + 1, y_max], and [z_midpoint + 1, z_max]
                new_sub_box_5 = [[box_x_range[0], box_x_range_midpoint], [box_y_range[0], box_y_range_midpoint], [box_z_range_midpoint + 1, box_z_range[1]]]
                new_sub_box_6 = [[box_x_range_midpoint + 1, box_x_range[1]], [box_y_range[0], box_y_range_midpoint], [box_z_range_midpoint + 1, box_z_range[1]]]
                new_sub_box_7 = [[box_x_range[0], box_x_range_midpoint], [box_y_range_midpoint + 1, box_y_range[1]], [box_z_range_midpoint + 1, box_z_range[1]]]
                new_sub_box_8 = [[box_x_range_midpoint + 1, box_x_range[1]], [box_y_range_midpoint + 1, box_y_range[1]], [box_z_range_midpoint + 1, box_z_range[1]]]

                new_sub_boxes = []
                new_sub_boxes.append(new_sub_box_1)
                new_sub_boxes.append(new_sub_box_2)
                new_sub_boxes.append(new_sub_box_3)
                new_sub_boxes.append(new_sub_box_4)
                new_sub_boxes.append(new_sub_box_5)
                new_sub_boxes.append(new_sub_box_6)
                new_sub_boxes.append(new_sub_box_7)
                new_sub_boxes.append(new_sub_box_8)
                    
                for new_sub_box in new_sub_boxes:
                    # checks if a sub-box is at least partially contained inside the user-specified box. if so, then the sub-box will be recursively searched 
                    # until an entire sub-box is inside the user-specified box
                    new_sub_box_partially_contained = self.boxesOverlap(new_sub_box, user_db_box)

                    if new_sub_box_partially_contained:
                        self.recursiveSubBoxesInFile(new_sub_box, user_db_box, sub_boxes)
        return
        
    def identifySubBoxesInFile(self, user_db_box, var, timepoint):
        # initially assumes the user-specified box in the file is not the entire box representing the file. the database file box will be sub-divided into
        # morton cubes until the user-specified box is completely mapped by all of these sub-cubes.
        user_db_box_x_range = user_db_box[0]
        user_db_box_y_range = user_db_box[1]
        user_db_box_z_range = user_db_box[2]
        
        user_db_box_x_min = user_db_box_x_range[0]
        user_db_box_y_min = user_db_box_y_range[0]
        user_db_box_z_min = user_db_box_z_range[0]
        
        # retrieve the morton index limits (minLim, maxLim) of the cube representing the whole database file
        f, cornercode, offset, minLim, maxLim = self.getFileForPoint(user_db_box_x_min, user_db_box_y_min, user_db_box_z_min, var, timepoint)
        minLim_xyz = self.mortoncurve.unpack(minLim)
        maxLim_xyz = self.mortoncurve.unpack(maxLim)
        
        # get the box for the entire database file so that it can be recursively broken down into cubes
        db_box = [[minLim_xyz[0], maxLim_xyz[0]], [minLim_xyz[1], maxLim_xyz[1]], [minLim_xyz[2], maxLim_xyz[2]]]
        
        # these are the constituent file sub-cubes that make up the part of the user-specified box in the database file
        sub_boxes = []
        self.recursiveSubBoxesInFile(db_box, user_db_box, sub_boxes)

        return sub_boxes
        
    def getVelocitiesForAllPoints(self, x_range, y_range, z_range, min_step = 1):
        # manually retrieves the velocities for all points inside the box. this is computationally expensive and not efficient, and this function is deprecated.
        x_min = x_range[0]; x_max = x_range[1];
        y_min = y_range[0]; y_max = y_range[1];
        z_min = z_range[0]; z_max = z_range[1];
        
        current_x_max = x_max
        current_y_max = y_max
        current_z_max = z_max
        
        velocity_map = {}
        velocity_data = np.array([-1, -1, -1])
        for x_point in np.arange(x_min, x_max + 1, min_step):
            for y_point in np.arange(y_min, y_max + 1, min_step):
                for z_point in np.arange(z_min, z_max + 1, min_step):
                    velocity_data = self.getISO_Point(x_point, y_point, z_point, var = 'vel', timepoint = 0, verbose = False)
                    
                    velocity_map[(x_point, y_point, z_point)] = velocity_data
                    #print(x_point)
                    #print(cornercode, offset)
        
        return velocity_map
        
    def getOffset(self, X, Y, Z):
        """
        TODO is this code correct for velocity as well?  YES
        """
        # morton curve index corresponding to the user specified X, Y, and Z values
        code = self.mortoncurve.pack(X, Y, Z)
        # always looking at an 8 x 8 x 8 box around the grid point, so the shift is always 9 bits to determine 
        # the bottom left corner of the box. the cornercode (bottom left corner of the 8 x 8 x 8 box) is always 
        # in the same file as the user-specified grid point.
        # equivalent to 512 * (math.floor(code / 512))
        cornercode = (code >> 9) << 9
        corner = np.array(self.mortoncurve.unpack(cornercode))
        # calculates the offset between the grid point and corner of the box and converts it to a 4-byte float.
        offset = np.sum((np.array([X, Y, Z]) - corner) * np.array([1, 8, 64]))
        
        return cornercode, offset
    
    def getFileForPoint_nocache(self, X, Y, Z, var = 'pr', timepoint = 0):
        """
        querying the SQL metadata database for the information the specific file corresponding to the user specified 
        X, Y, and Z grid point. this method is slow, and is deprecated.
        """
        cornercode, offset = self.getOffset(X, Y, Z)
        
        sql = f"""
        select dbm.ProductionMachineName
        , dbm.ProductionDatabaseName
        , dbm.minLim, dbm.maxLim
        , dbm.minTime, dbm.maxTime
        , dp.path
        from databasemap dbm
           join datapath{str(self.N) + self.N_subtitle} dp
             on dp.datasetid=dbm.datasetid
           and dp.productionmachinename=dbm.productionmachinename
           and dp.ProductionDatabaseName=dbm.ProductionDatabaseName
        where dbm.datasetname = 'isotropic{str(self.N) + self.N_subtitle}'
        and {cornercode} between minLim and maxLim
        order by minlim
        """
        df = cj.executeQuery(sql, "turbinfo")
        t = df.loc[0]
        dataN = t.path.split("/")
        f = f'/home/idies/workspace/turb/data{t.ProductionMachineName[-2:]}_{dataN[2][-2:]}/{dataN[-1]}/{t.ProductionDatabaseName}_{var}_{timepoint}.bin'
        return f, cornercode, offset, t.minLim
    
    def getFileForPoint(self, X, Y, Z, var = 'pr', timepoint = 0):
        """
        querying the cached SQL metadata for the file for the user specified grid point
        """
        cornercode, offset = self.getOffset(X, Y, Z)
        t = self.cache[(self.cache['minLim'] <= cornercode) & (self.cache['maxLim'] >= cornercode)]
        t = t.iloc[0]
        dataN = t.path.split("/")
        f = f'/home/idies/workspace/turb/data{t.ProductionMachineName[-2:]}_{dataN[2][-2:]}/{dataN[-1]}/{t.ProductionDatabaseName}_{var}_{timepoint}.bin'
        return f, cornercode, offset, t.minLim, t.maxLim
        
    def getISO_Point(self, X, Y, Z, var = 'pr', timepoint = 0, verbose = False):
        """
        find the value for the specified var(iable) at the specified grid point X, Y, Z and the specified timepoint. position 
        is assumed to be a point of the grid, i.e. should be integers, and timepoint should be an integer between 0 and 5.
        """
        f, cornercode, offset, minLim, maxLim = self.getFileForPoint(X, Y, Z, var, timepoint)
        if verbose:
            print(f, cornercode, offset, minLim, maxLim)
            print(self.mortoncurve.unpack(minLim))
            print(self.mortoncurve.unpack(maxLim))
            
        # currently two vars are accepted (pressure and velocity). both are 4 byte values, but there are 3 velocities 
        # per grid point (velocity magnitude along each dimension). 
        N = 1
        if var == 'vel':
            N = 3
        
        seek_distance = N * 4 * (cornercode + offset - minLim)
        with open(f, 'rb') as b:
            b.seek(seek_distance)
            xraw = b.read(N * 4)
            
        l = struct.unpack('f' * N, xraw)
        return np.asarray(l)

In [19]:
# gets velocity for all points inside the user specified box.
iso_data = IsoCube(cube_num = 8192, cube_dimensions = 3, cube_subtitle = '')

# user specified box rather than a singular data point.
x_range = [1019, 1023]
y_range = [100, 100]
z_range = [1000, 1023]

# test values for the whole grid.
# x_range = [0, 34]
# y_range = [0, 511]
# z_range = [0, 511]

# variable of interest, currently set to velocity.
var = 'vel'

# time point
timepoint = 0

# get a map of the database files where all the data points are in.
%time user_single_db_boxes = iso_data.identifySingleDatabaseFileSubBoxes(x_range, y_range, z_range, var, timepoint)
# recursively break down each single file box into sub-boxes, each of which is exactly one of the sub-divided cubes of the database file.
sub_db_boxes = {}
for db_file in sorted(user_single_db_boxes, key = lambda x: os.path.basename(x)):
    user_db_box = user_single_db_boxes[db_file]
    
    %time sub_boxes = iso_data.identifySubBoxesInFile(user_db_box, var, timepoint)
    
    sub_db_boxes[db_file] = sub_boxes

print(f'-\nnumber of db files that the user-specified box is found in = {len(user_single_db_boxes)}\n-')
for db_file in sorted(user_single_db_boxes, key = lambda x: os.path.basename(x)):
    print(db_file)
    print(user_single_db_boxes[db_file])

print('-\nsub-boxes found in the database file:\n-')
for db_file in sorted(sub_db_boxes, key = lambda x: os.path.basename(x)):
    print(db_file)
    print(f'number of boxes = {len(sub_db_boxes[db_file])}')
    #print(sub_db_boxes[db_file])
    
# read a file for a point.
# print('-')
# X = 512
# Y = 100
# Z = 3
#
# %time var_data = iso_data.getISO_Point(X, Y, Z, var, timepoint, verbose = True)
#
# print(f'-\n{var} for point ({X}, {Y}, {Z}) = {var_data}')
    
# read the velocities for all points in the box by brute force.
# %time velocities = iso_data.getVelocitiesForAllPoints(x_range, y_range, z_range, min_step = 1)
#
# print(f'\nnum velocities = {len(v3)}\n-')
# for velocity_point in velocities:
#     print(velocity_point)
#     print(velocities[velocity_point])
#     print('-')

CPU times: user 11.4 ms, sys: 0 ns, total: 11.4 ms
Wall time: 11.4 ms
CPU times: user 7.89 ms, sys: 997 µs, total: 8.89 ms
Wall time: 8.69 ms
-
number of db files that the user-specified box is found in = 1
-
/home/idies/workspace/turb/data09_01/iso8192db_06/iso8192db0006_vel_0.bin
([1019, 1023], [100, 100], [1000, 1023])
-
sub-boxes found in the database file:
-
/home/idies/workspace/turb/data09_01/iso8192db_06/iso8192db0006_vel_0.bin
number of boxes = 72


In [71]:
# range for X, Y, and Z grid points are 0 to N-1
X = 1053; Y = 100; Z = 1000;
# variable of interest, currently set to velocity
var = 'vel'
# time point
timepoint = 0
%time iso_data.getFileForPoint(X, Y, Z, var, timepoint)
%time iso_data.getFileForPoint_nocache(X, Y, Z, var, timepoint)

CPU times: user 3.86 ms, sys: 0 ns, total: 3.86 ms
Wall time: 3.75 ms
CPU times: user 28.8 ms, sys: 8.01 ms, total: 36.8 ms
Wall time: 1.22 s


('/home/idies/workspace/turb/data05_03/iso8192db_13/iso8192db0013_vel_0.bin',
 1687886336,
 37,
 1610612736)

In [43]:
# for trials on  http://turbulence.pha.jhu.edu/webquery/query.aspx
# converts the X, Y, and Z points to the domain of [0, 2*pi]
dxyz=2*math.pi/8192
x=X*dxyz
y=Y*dxyz
z=Z*dxyz
# enter these values in UI
x,y,z

(0.42414568785037976, 0.07669903939428206, 0.7669903939428205)

In [3]:
# gets velocity for all points inside the user specified box
iso_data = IsoCube(cube_num = 8192, cube_dimensions = 3, cube_subtitle = '')

var = 'vel'
timepoint = 0

# user specified box rather than a singular data point
x_range = [1000, 1023]
y_range = [100, 100]
z_range = [1022, 1023]

x_range = [0, 511]
y_range = [0, 10]
z_range = [0, 0]

start_time = time.perf_counter()

%time iso_data.mapBoxPointsToBytes(x_range, y_range, z_range, var, timepoint, verbose = False)
    
end_time = time.perf_counter()

print(f'total time = {round(end_time - start_time, 5)} seconds')    

5632
CPU times: user 1.63 s, sys: 2.35 ms, total: 1.63 s
Wall time: 1.63 s
total time = 1.63346 seconds


## compare direct access to cutout
To get raw data in HDF5 format one can run a job at http://turbulence.idies.jhu.edu/cutout/jobs.
Result will be put on scratch. Here an example reading the result of such a job, using the parameters.txt to find the location.

In [44]:
import h5py
import json

In [180]:
# ran job x in [1000,1010], y and z in [1,10]
#folder='/home/idies/workspace/Temporary/gerard/scratch/jobs/__turbcutout__/20211012/20211012094603-148997/'
folder = '/home/idies/workspace/Temporary/mschnau1/scratch/jobs/__turbcutout__/20211027/20211027121934-151320/'
p=f'{folder}parameters.txt' 
with open(p,'r') as f:
    pars=json.load(f)
pars

{'dataset': 'isotropic8192',
 'filter_width': 1,
 'function': 'u',
 'output_filename': 'isotropic8192',
 'stridet': 1,
 'stridex': 1,
 'stridey': 1,
 'stridez': 1,
 'te': 1,
 'token': 'edu.jhu.pha.turbulence.testing-201406',
 'ts': 1,
 'xe': 1010,
 'xs': 1000,
 'ye': 10,
 'ys': 1,
 'ze': 10,
 'zs': 1}

In [181]:
f=f'{folder}isotropic8192.h5' 
h5=h5py.File(f,'r')

In [182]:
h5['Velocity_0001'].shape

(10, 10, 11, 3)

In [183]:
x=h5['xcoor']
y=h5['ycoor']
z=h5['zcoor']
vel=h5['Velocity_0001']

In [184]:
# choose an offset in the retrieved cutout
dx=3;dy=4;dz=7;
v1=vel[dz,dy,dx,:]
v1

array([-2.850525  ,  0.42863464,  3.3042493 ], dtype=float32)

In [185]:
# calculate position in the file
X=pars['xs']+dx-1
Y=pars['ys']+dy-1  # cutout starts at 1
Z=pars['zs']+dz-1

In [186]:
var='vel'
timepoint=pars['ts']-1   # in cutout time starts at 1
v2=isoData.getISO_Point(X,Y,Z,var,timepoint,verbose=False)
v2

array([-2.8505249 ,  0.42863464,  3.30424929])

In [187]:
# hope this would be zeros
v1-v2

array([0., 0., 0.])

In [188]:
v4=isoData.getISO_Point(553,100,1000,var,timepoint,verbose=False)
v4

array([0.88515317, 2.20288801, 2.40113282])

In [206]:
print(v3)
v4-v3[X, Y, Z]

{(553, 100, 1000): array([0.88515317, 2.20288801, 2.40113282])}


array([0., 0., 0.])